In [ ]:


import pandas as pd

bronx_df = pd.read_csv('bronx_df(in).csv')
brooklyn_df = pd.read_csv('brooklyn_df(in).csv')
manhattan_df = pd.read_csv('manhattan_df(in).csv')
queens_df = pd.read_csv('queens_df(in).csv')
staten_island_df = pd.read_csv('staten_island_df(in).csv')

In [ ]:
# combines all borough datasets into one
yelp_combined_df = pd.concat([bronx_df, brooklyn_df, manhattan_df, queens_df, staten_island_df], ignore_index=True)

# removes Location Point1 column
if 'Location Point1' in yelp_combined_df.columns:
    yelp_combined_df = yelp_combined_df.drop(columns=['Location Point1'])

# saves in file for download
yelp_combined_df.to_csv('boroughs_combined.csv', index=False)

In [ ]:
print(yelp_combined_df.head())

      CAMIS                                           DBA   BORO BUILDING  \
0  50159852                          MEJIA DELI FOOD CORP  Bronx     1201   
1  41124419                                    BURGER HUT  Bronx   152112   
2  41584120                                      7 SPICES  Bronx   906908   
3  50099398  LECHONERA POLLO SABROSO DEL VALLE RESTAURANT  Bronx     650B   
4  41308105                                EMILIO'S PIZZA  Bronx       80   

                 STREET  ZIPCODE       PHONE CUISINE DESCRIPTION  \
0            LELAND AVE      NaN  9174716834                 NaN   
1    WESTCHESTER AVENUE      NaN  7188426426            American   
2          E GUNHILL RD      NaN  3473268646           Caribbean   
3    WESTCHESTER AVENUE      NaN  3472977545             Spanish   
4  WEST KINGSRIDGE ROAD      NaN  7185633665               Pizza   

  INSPECTION DATE                                           ACTION  ...  \
0        1/1/1900                                    

In [ ]:
boroughs_combined_df = pd.read_csv('boroughs_combined.csv')

In [ ]:
#checks for duplicates
boroughs_combined_df = boroughs_combined_df.drop_duplicates()

In [ ]:
#counts amount of rows before cutting
rows_before = boroughs_combined_df.shape[0]

#drop row if key columns are empty or invalid
key_columns = ['DBA', 'BORO', 'yelp_rating', 'Latitude', 'Longitude', 'yelp_review_count']
boroughs_combined_df = boroughs_combined_df.dropna(subset=key_columns)

In [ ]:
#shows amount of rows that were dropped
rows_after = boroughs_combined_df.shape[0]
rows_dropped = rows_before - rows_after
print(f"Number of rows dropped in validation of columns: {rows_dropped}")

Number of rows dropped in validation of columns: 253


In [ ]:
#normalizing text data by converting to lower case, stripping spaces and removeal of special characters
boroughs_combined_df['DBA'] = boroughs_combined_df['DBA'].str.lower().str.strip()
boroughs_combined_df['BORO'] = boroughs_combined_df['BORO'].str.lower().str.strip()

In [ ]:
boroughs_combined_df['yelp_rating'] = pd.to_numeric(boroughs_combined_df['yelp_rating'], errors='coerce')
boroughs_combined_df['yelp_review_count'] = pd.to_numeric(boroughs_combined_df['yelp_review_count'], errors='coerce')

In [ ]:
boroughs_combined_df['yelp_rating'].fillna(0, inplace=True)
boroughs_combined_df['yelp_review_count'].fillna(0, inplace=True)

<ipython-input-11-8615ffa000e2>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  boroughs_combined_df['yelp_rating'].fillna(0, inplace=True)
<ipython-input-11-8615ffa000e2>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)

In [ ]:
lat_min, lat_max = 40.477399, 40.917577
long_min, long_max = -74.259090, -73.700272

boroughs_combined_df = boroughs_combined_df[
    (boroughs_combined_df['Latitude'].between(lat_min, lat_max)) &
    (boroughs_combined_df['Longitude'].between(long_min, long_max))
]


In [ ]:
missing_data_summary = boroughs_combined_df.isnull().sum()
print("\nMissing Data Summary:")
print(missing_data_summary)


Missing Data Summary:
CAMIS                        0
DBA                          0
BORO                         0
BUILDING                    25
STREET                       0
ZIPCODE                      0
PHONE                        3
CUISINE DESCRIPTION       2593
INSPECTION DATE              0
ACTION                    2593
VIOLATION CODE            2883
VIOLATION DESCRIPTION     2883
CRITICAL FLAG                0
SCORE                     3294
GRADE                    10791
GRADE DATE               12291
RECORD DATE                  0
INSPECTION TYPE           2593
Latitude                     0
Longitude                    0
Community Board             40
Council District            40
Census Tract                40
BIN                        185
BBL                         40
NTA                         40
yelp_rating                  0
yelp_review_count            0
dtype: int64


In [ ]:
review_count_threshold = 5000
outliers = boroughs_combined_df[boroughs_combined_df['yelp_review_count'] > review_count_threshold]
print(f"\nNumber of outliers in Yelp review count: {len(outliers)}")
print("Sample outliers:")
print(outliers[['DBA', 'yelp_review_count']].head())


Number of outliers in Yelp review count: 0
Sample outliers:
Empty DataFrame
Columns: [DBA, yelp_review_count]
Index: []


In [ ]:
inspection_data = pd.read_csv('DOHMH_New_York_City_Restaurant_Inspection_Results_20241203.csv')



<ipython-input-15-6aabe0ecf7cc>:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  inspection_data = pd.read_csv('DOHMH_New_York_City_Restaurant_Inspection_Results_20241203.csv')


In [ ]:
inspection_data['DBA'] = inspection_data['DBA'].str.lower().str.strip()
inspection_data['BORO'] = inspection_data['BORO'].str.lower().str.strip()

In [ ]:
inspection_data['CAMIS'] = inspection_data['CAMIS'].astype(str)
boroughs_combined_df['CAMIS'] = boroughs_combined_df['CAMIS'].astype(str)

In [ ]:
merged_data = pd.merge(
    boroughs_combined_df,
    inspection_data,
    on='CAMIS',
    how='inner',
    suffixes=('_yelp', '_inspection')
)

In [ ]:
output_path = '/content/cleaned_merged_inspection_data.csv'
merged_data.to_csv(output_path, index=False)
print(f"\nFinal cleaned and merged dataset saved to: {output_path}")


Final cleaned and merged dataset saved to: /content/cleaned_merged_inspection_data.csv


In [ ]:
print("\nFinal Merged Dataset Preview:")
print(merged_data.head())


Final Merged Dataset Preview:
      CAMIS                    DBA_yelp BORO_yelp BUILDING_yelp  \
0  50126924  walnut bus stop restaurant     bronx           881   
1  50126924  walnut bus stop restaurant     bronx           881   
2  50126924  walnut bus stop restaurant     bronx           881   
3  50126924  walnut bus stop restaurant     bronx           881   
4  50126924  walnut bus stop restaurant     bronx           881   

        STREET_yelp  ZIPCODE_yelp  PHONE_yelp CUISINE DESCRIPTION_yelp  \
0  EAST  134 STREET       10454.0  9177741138                 American   
1  EAST  134 STREET       10454.0  9177741138                 American   
2  EAST  134 STREET       10454.0  9177741138                 American   
3  EAST  134 STREET       10454.0  9177741138                 American   
4  EAST  134 STREET       10454.0  9177741138                 American   

  INSPECTION DATE_yelp                                      ACTION_yelp  ...  \
0            4/10/2023  Violations were c

In [ ]:
print("\nFinal Merged Dataset Info:")
print(merged_data.info())


Final Merged Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208811 entries, 0 to 208810
Data columns (total 54 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   CAMIS                             208811 non-null  object 
 1   DBA_yelp                          208811 non-null  object 
 2   BORO_yelp                         208811 non-null  object 
 3   BUILDING_yelp                     208698 non-null  object 
 4   STREET_yelp                       208811 non-null  object 
 5   ZIPCODE_yelp                      208811 non-null  float64
 6   PHONE_yelp                        208808 non-null  object 
 7   CUISINE DESCRIPTION_yelp          206079 non-null  object 
 8   INSPECTION DATE_yelp              208811 non-null  object 
 9   ACTION_yelp                       206079 non-null  object 
 10  VIOLATION CODE_yelp               203684 non-null  object 
 11  VIOLATION DESCRIPTION_ye